# Run parcel tracking with tracer

In [1]:
import numpy as np
from datetime import timedelta as delta, datetime
import xarray as xr
from parcels import FieldSet, Variable, ParticleSet, JITParticle, AdvectionRK4_3D, ErrorCode
import glob

%matplotlib inline

#### Parameters

In [2]:
year = 2002

# Sub-domain dimensions:
jmin, jmax = 159, 799
imin, imax = 1139, 2179

#### General setup

Timestamps:

In [3]:
# Mesh mask
mesh_mask = '/ocean/brogalla/GEOTRACES/data/ANHA12/ANHA12_mesh1.nc'

# Lists of ANHA12 NEMO U,V,W files (on full ANHA12 domain)
folder_ANHA12      = '/data/brogalla/ANHA12/'
ANHA12_gridU_files = sorted(glob.glob(f'{folder_ANHA12}ANHA12-EXH006_y{year}m??d??_gridU.nc'))[0:20]
ANHA12_gridV_files = sorted(glob.glob(f'{folder_ANHA12}ANHA12-EXH006_y{year}m??d??_gridV.nc'))[0:20]
ANHA12_gridW_files = sorted(glob.glob(f'{folder_ANHA12}ANHA12-EXH006_5d_gridW_y{year}m??d??.nc'))[0:20]

# Lists of ANHA12 NEMO Pb tracer files (on sub-domain of full ANHA12 domain)
folder_Pb = f'/data/brogalla/run_storage/Pb-new-ini2-20220922/'
files_Pb  = sorted(glob.glob(f'{folder_Pb}ANHA12_EXH006_5d_{year}0101_{year}1231_ptrc_T_{year}*'))[0:20]

# Create timestamps from file dates: (because of time_origin issue in ANHA12 netcdf dynamics files)
time_stamps_files = []
for n, file in enumerate(ANHA12_gridU_files):
    dateU  = datetime.strptime(file[36:47],'y%Ym%md%d')
    dateV  = datetime.strptime(ANHA12_gridV_files[n][36:47],'y%Ym%md%d')
    dateW  = datetime.strptime(ANHA12_gridW_files[n][45:56],'y%Ym%md%d')
    datePb = datetime.strptime(files_Pb[n][99:107],'%Y%m%d')
    
    # double-check that the date is the same for all files
    if (dateU != dateV) | (dateU != dateW) | (dateU != datePb):
        print('Dates not equal!')
    else:   
        time_stamps_files.append(np.datetime64(f'{dateU.year:02}-{dateU.month:02}-{dateU.day:02}'))

# array of file timestamps
timestamps_files = np.expand_dims(np.array(time_stamps_files), axis=1)

Fieldset:

In [4]:
filenames = {'U': {'lon': mesh_mask   , 'lat': mesh_mask  , 'depth': ANHA12_gridW_files[0], 'data': ANHA12_gridU_files},
             'V': {'lon': mesh_mask   , 'lat': mesh_mask  , 'depth': ANHA12_gridW_files[0], 'data': ANHA12_gridV_files},
             'W': {'lon': mesh_mask   , 'lat': mesh_mask  , 'depth': ANHA12_gridW_files[0], 'data': ANHA12_gridW_files},
             'Pb': {'lon': files_Pb[0], 'lat': files_Pb[0], 'depth': files_Pb[0], 'data': files_Pb}}

dimensions = {'U': {'lon': 'nav_lon', 'lat': 'nav_lat', 'depth': 'depthw'},
              'V': {'lon': 'nav_lon', 'lat': 'nav_lat', 'depth': 'depthw'},
              'W': {'lon': 'nav_lon', 'lat': 'nav_lat', 'depth': 'depthw'},
              'Pb': {'lon': 'nav_lon', 'lat': 'nav_lat', 'depth': 'deptht'}}

variables = {'U': 'vozocrtx',
             'V': 'vomecrty',
             'W': 'vovecrtz',
             'Pb': 'dissolpb'}

# subset U,V,W indices because they are on a larger grid than the Pb model grid (same resolution)
indices_subset = {'U':  {'lon': range(jmin, jmax), 'lat': range(imin, imax), 'depth': range(0,50)},
                  'V':  {'lon': range(jmin, jmax), 'lat': range(imin, imax), 'depth': range(0,50)},
                  'W':  {'lon': range(jmin, jmax), 'lat': range(imin, imax), 'depth': range(0,50)},
                  'Pb': {'lon': range(0,640)     , 'lat': range(0,1040)    , 'depth': range(0,50)}}

# needed to provide time as timestamps in order to avoid time origin type issue
fieldset  = FieldSet.from_nemo(filenames, variables, dimensions, \
                               timestamps=timestamps_files, \
                               allow_time_extrapolation=False, \
                               indices=indices_subset)

#### Classes and functions

In [5]:
# define new particle class for Pb
class PbParticle(JITParticle): 
    dissolvedPb = Variable('dissolpb', initial=fieldset.Pb)  # Variable 'dissolpb' initialised by sampling the dissolved Pb field

In [6]:
def DeleteParticle(particle, fieldset, time):
    print("Particle [%d] lost !! (%g %g %g %g)" % (particle.id, particle.lon, particle.lat, particle.depth, particle.time))
    particle.delete()

In [7]:
def SamplePb(particle, fieldset, time):
    particle.dissolpb = fieldset.Pb[particle.time, particle.depth, particle.lat, particle.lon]

#### Run

Specify particle set with starting locations:

In [8]:
release_time  = 0
release_lon   = [-130, -145, -70, -55] # east
release_lat   = [75, 74, 75, 65]       # north
pset = ParticleSet(fieldset=fieldset, pclass=PbParticle, lon=release_lon, lat=release_lat, time=0)

Run with Pb sampling kernel:

In [9]:
output_file = pset.ParticleFile(name="/ocean/brogalla/GEOTRACES/parcels/Pb-chapter/Pb-test2.zarr", outputdt=delta(hours=12))

pset.execute(AdvectionRK4_3D + pset.Kernel(SamplePb), 
             runtime     = delta(hours=24*10), 
             dt          = delta(hours=2),
             output_file = output_file,
             recovery    = {ErrorCode.ErrorOutOfBounds: DeleteParticle})

INFO: Compiled ArrayPbParticleAdvectionRK4_3DSamplePb ==> /tmp/parcels-2672/lib7f8843b28205f099074a7d6f4fa10a04_0.so


Particle [0] lost !! (-130 75 0.494025 0)
Particle [1] lost !! (-145 74 0.494025 0)
Particle [3] lost !! (-55 65 0.494025 0)


INFO: Output files are stored in /ocean/brogalla/GEOTRACES/parcels/Pb-chapter/Pb-test2.zarr.
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 864000.0/864000.0 [00:05<00:00, 152710.81it/s]


Most, but not all particles are lost when running with the sampling kernel.

Without the Pb sampling kernel:

In [10]:
output_file = pset.ParticleFile(name="/ocean/brogalla/GEOTRACES/parcels/Pb-chapter/Pb-test2.zarr", outputdt=delta(hours=12))

pset.execute(AdvectionRK4_3D, 
             runtime     = delta(hours=24*10), 
             dt          = delta(hours=2),
             output_file = output_file,
             recovery    = {ErrorCode.ErrorOutOfBounds: DeleteParticle})


INFO: Compiled ArrayPbParticleAdvectionRK4_3D ==> /tmp/parcels-2672/liba3b471138d57e26ed5f9d686a2a025f3_0.so
INFO: Output files are stored in /ocean/brogalla/GEOTRACES/parcels/Pb-chapter/Pb-test2.zarr.
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 864000.0/864000.0 [00:02<00:00, 319226.05it/s]


The particles do not throw an "ErrorOutOfBounds" when released at the same locations without sampling the Pb field, so the issue relates to the Pb tracer field. 

#### Check some of the fieldset values

In [12]:
# U, V, W all have the same values
print(np.amin(fieldset.U.lon), np.amax(fieldset.U.lon))
print(np.amin(fieldset.Pb.lon), np.amax(fieldset.Pb.lon))

print(np.amin(fieldset.U.lat), np.amax(fieldset.U.lat))
print(np.amin(fieldset.Pb.lat), np.amax(fieldset.Pb.lat))

print(np.amin(fieldset.U.depth), np.amax(fieldset.U.depth))
print(np.amin(fieldset.Pb.depth), np.amax(fieldset.Pb.depth))

-176.72366 -32.57282
-176.72366 -32.57282
55.4229 85.85598
55.4229 85.85598
0.0 5500.0015
0.49402538 5727.9165


double-checked that the time is being read in correctly; subset indexing seems to be correct when I look at the min and max of lon and lat